In [45]:
import time as time
from urllib.parse import urlencode, urlparse
import requests

In [46]:
def get_jsons(date):
    base_url = "http://93.183.71.44/agro/api/v1/device-initial-list"
    params = {"from_datetime": date}
    # Полный URL с параметрами
    url = f"{base_url}?{urlencode(params)}"
    print("Отправка запроса на:", url)
    try:
        response = requests.get(url, timeout=10)
        
        # Проверяем статус ответа
        if response.status_code == 200:
            print("Успешный ответ:")
            print(len(response.json()), " строк")
            if len(response.json()) > 0:
                return response.json()
            else:
                return False
        else:
            print(f"Ошибка: {response.status_code}")
            print(response.text)
            return False

    except requests.exceptions.RequestException as e:
        print("Ошибка при выполнении запроса:", e)


In [47]:
jsons= get_jsons('02:02:34 04.09')
# "2025-09-04T02:02:33"

Отправка запроса на: http://93.183.71.44/agro/api/v1/device-initial-list?from_datetime=02%3A02%3A34+04.09
Успешный ответ:
17492  строк


In [48]:
def transform_data(data):
    return {
        "sens": data["sens"],
        "port_1": {
            "hum_air": data["meteo"]["humidity"],
            "pressure_hpa": data["meteo"]["pressure_hpa"],
            "pressure_mmhg": data["meteo"]["pressure_mmhg"],
            "wind_direction": data["meteo"]["wind_direction"],
            "wind_speed": data["meteo"]["wind_speed"],
            "wind_gust": data["meteo"]["wind_gust"],
            "uv_intensity": data["meteo"]["uv_intensity"],
            "uv_index": data["meteo"]["uv_index"],
            "illuminance": data["meteo"]["illuminance"],
            "rainfall": data["meteo"]["rainfall"],
            "temp_air": data["meteo"]["temperature"]
        },
        "port_2": {
            "hum_soil_10": data["soil"]["moisture"],
            "temp_soil_10": data["soil"]["temperature"]
        }
    }

In [50]:
import json
def write_jsonl(data_list, filepath):
    with open(filepath, 'w', encoding='utf-8') as f:
        for item in data_list:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')

In [49]:
q=[]
for i in range(len(jsons)):
    if jsons[i]['sens']['id_s'] in ['7','5','10']:
        q.append(transform_data(jsons[i]))
print(len(q))

13472


In [51]:
write_jsonl(q,'data.json')

In [10]:
def fetch_sensor_data(start_date: str):
    base_url = "http://93.183.71.105:8802/api/v1/module-data"
    params = {
        "sensor_ids": [5, 7, 10, 2],
        "start_date": start_date,
        "end_date": "2025-12-23T00:00:00"
    }
    # urlencode с параметром doseq=True корректно обрабатывает списки
    query_string = urlencode(params, doseq=True)
    url = f"{base_url}?{query_string}"
    response = requests.get(url)
    response.raise_for_status()
    return response.json()

In [30]:
from datetime import date, timedelta

def generate_past_dates(days: int):
    return (date.today() - timedelta(days=days)).isoformat()

In [34]:
pac=[]
shift=0
while True: 
    d=generate_past_dates(shift)
    pac=fetch_sensor_data(str(d)+"T00:00:00")
    # print(pac)
    if len(pac)!=0:break
    else:
        shift+=1
    print(d)
last_time=pac[-1]['ts']
    

2025-10-30
2025-10-29
2025-10-28
2025-10-27
2025-10-26
2025-10-25
2025-10-24
2025-10-23
2025-10-22
2025-10-21
2025-10-20
2025-10-19
2025-10-18
2025-10-17
2025-10-16
2025-10-15
2025-10-14
2025-10-13
2025-10-12
2025-10-11
2025-10-10
2025-10-09
2025-10-08
2025-10-07
2025-10-06
2025-10-05
2025-10-04
2025-10-03
2025-10-02
2025-10-01
2025-09-30
2025-09-29
2025-09-28
2025-09-27
2025-09-26
2025-09-25
2025-09-24
2025-09-23
2025-09-22
2025-09-21
2025-09-20
2025-09-19
2025-09-18
2025-09-17
2025-09-16
2025-09-15
2025-09-14
2025-09-13
2025-09-12
2025-09-11
2025-09-10
2025-09-09
2025-09-08
2025-09-07
2025-09-06
2025-09-05


In [35]:
last_time

'2025-09-04T21:43:08'

In [43]:
lt=last_time[-8:]+' '+last_time[8:10]+'.'+last_time[5:7]

In [44]:
jsons= get_jsons(lt)

Отправка запроса на: http://93.183.71.44/agro/api/v1/device-initial-list?from_datetime=21%3A43%3A08+04.09
Успешный ответ:
17173  строк


In [50]:
import json
import time
import paho.mqtt.client as mqtt
import logging


In [57]:
# Логирование
def sender(data_list):
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger(__name__)

    # Настройки MQTT
    BROKER = "93.183.71.105"
    PORT = 1883
    DEVICE_SERIAL = "00000001"
    USERNAME = None
    PASSWORD = None

    MQTT_TOPIC_DATA = f"mqtt/devices/{DEVICE_SERIAL}/data"

    # Callback
    def on_connect(client, userdata, flags, rc, properties=None):
        if rc == 0:
            logger.info("Connected to MQTT Broker")
        else:
            logger.error(f"Failed to connect, rc={rc}")

    def on_publish(client, userdata, mid, properties=None):
        logger.info(f"Message {mid} published")

    # Создаём клиент и подключаемся
    client = mqtt.Client(client_id=DEVICE_SERIAL)
    if USERNAME:
        client.username_pw_set(USERNAME, PASSWORD)

    client.on_connect = on_connect
    client.on_publish = on_publish

    client.connect(BROKER, PORT, keepalive=60)
    client.loop_start()

    # Публикуем каждую запись по очереди

    for entry in data_list:
        payload = json.dumps(entry)
        client.publish(MQTT_TOPIC_DATA, payload=payload, qos=1)
        logger.info(f"Published: {payload}")
        time.sleep(0.01)  # пауза между публикациями

    client.loop_stop()
    client.disconnect()
    logger.info("Disconnected from MQTT")

In [58]:
sender(q)

C:\Users\nik\AppData\Local\Temp\ipykernel_8936\3673380718.py:26: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client(client_id=DEVICE_SERIAL)
INFO:__main__:Published: {"sens": {"id_s": "5", "batt": 97, "rssi": -80, "time": "23:57:15 01.09"}, "port_1": {"hum_air": 67.0, "pressure_hpa": 981.9, "pressure_mmhg": 736.49, "wind_direction": "\u042e", "wind_speed": 1.0, "wind_gust": 2.0, "uv_intensity": 0.0, "uv_index": 0.0, "illuminance": 0.0, "rainfall": 2.03, "temp_air": 22.3}, "port_2": {"hum_soil_10": 14.1, "temp_soil_10": 28.4}}


INFO:__main__:Published: {"sens": {"id_s": "10", "batt": 91, "rssi": -104, "time": "00:01:55 02.09"}, "port_1": {"hum_air": 69.0, "pressure_hpa": 983.7, "pressure_mmhg": 737.84, "wind_direction": "\u042e\u0417", "wind_speed": 0.0, "wind_gust": 0.0, "uv_intensity": 0.0, "uv_index": 0.0, "illuminance": 0.0, "rainfall": 4.57, "temp_air": 21.3}, "port_2": {"hum_soil_10": 13.9, "temp_soil_10": 28.3}}
INFO:__main__:Published: {"sens": {"id_s": "7", "batt": 117, "rssi": -103, "time": "00:02:03 02.09"}, "port_1": {"hum_air": 69.0, "pressure_hpa": 986.1, "pressure_mmhg": 739.64, "wind_direction": "\u042e\u0412", "wind_speed": 0.0, "wind_gust": 1.0, "uv_intensity": 0.0, "uv_index": 0.0, "illuminance": 0.0, "rainfall": 2.79, "temp_air": 21.8}, "port_2": {"hum_soil_10": 15.9, "temp_soil_10": 27.4}}
INFO:__main__:Published: {"sens": {"id_s": "5", "batt": 97, "rssi": -80, "time": "00:12:30 02.09"}, "port_1": {"hum_air": 66.0, "pressure_hpa": 982.0, "pressure_mmhg": 736.56, "wind_direction": "\u042e"

KeyboardInterrupt: 

INFO:__main__:Connected to MQTT Broker
INFO:__main__:Message 783 published
INFO:__main__:Message 784 published
INFO:__main__:Message 785 published
INFO:__main__:Message 786 published
INFO:__main__:Message 787 published
INFO:__main__:Message 788 published
INFO:__main__:Message 789 published
INFO:__main__:Message 790 published
INFO:__main__:Message 791 published
INFO:__main__:Message 792 published
INFO:__main__:Message 793 published
INFO:__main__:Message 794 published
INFO:__main__:Message 795 published
INFO:__main__:Message 796 published
INFO:__main__:Message 797 published
INFO:__main__:Message 798 published
INFO:__main__:Message 799 published
INFO:__main__:Message 800 published
INFO:__main__:Message 801 published
INFO:__main__:Message 802 published
INFO:__main__:Message 803 published
INFO:__main__:Message 804 published
INFO:__main__:Message 805 published
INFO:__main__:Message 806 published
INFO:__main__:Message 807 published
INFO:__main__:Message 808 published
INFO:__main__:Message 809